In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = './model'
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained(model_path)
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)

In [5]:
article = """
A frontal boundary that had been stalled over the mid-Atlantic had lifted north of the region by the morning of 
June 29. Later that day and into the evening, the front once again approached, this time as a strong cold front,
as low pressure tracked through New England and began to intensify offshore in the Gulf of Maine. 
The combination of strong frontal forcing and a warm, unstable environment ahead of the front led to 
widespread severe thunderstorms developing. Numerous reports of damaging wind, as well as some hail, 
were received in association with these storms.
"""

In [6]:
# encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)

In [7]:
# generate the summarization output
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,     3,     9,   851,    24,   141,   118,     3,  8407,  1361,
           147,     8,  2076,    18,   188,    17,  1618,  1225, 19464,  3457,
            13,     8,  1719,     3,     5,   865,    24,   239,    11,   139,
             8,  2272,     6,     8,   851,   728,   541, 15319,     3,     5,
           731,  1666, 22679,   190,   126,  2789,    11,  1553,    12,  9608,
          4921, 16667,    16,     8, 13566,    13, 13905,     3,     5,     1]])
<pad> a front that had been stalled over the mid-Atlantic lifted north of the region. later that day and into the evening, the front once again approached. low pressure tracked through new England and began to intensify offshore in the Gulf of Maine.</s>


In [9]:
import json
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoConfig

def serverless_pipeline(model_path='./model'):
    # initialize the model architecture and weights
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    # initialize the model tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    def predict(context):
        """predicts the answer on an given question and context. Uses encode and decode method from above"""
        # encode the text into tensor of integers using the appropriate tokenizer
        inputs = tokenizer.encode("summarize: " + context, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(
                inputs, 
                max_length=150, 
                min_length=40, 
                length_penalty=2.0, 
                num_beams=4, 
                early_stopping=True)
        answer = tokenizer.decode(outputs[0])
        return answer
    return predict

# initializes the pipeline
question_answering_pipeline = serverless_pipeline()

def handler(event, context):
    try:
        # loads the incoming event into a dictonary
        body = json.loads(event['body'])
        # uses the pipeline to predict the answer
        answer = question_answering_pipeline(context=body['context'])
        return {
            "statusCode": 200,
            "headers": {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*',
                "Access-Control-Allow-Credentials": True

            },
            "body": json.dumps({'answer': answer})
        }
    except Exception as e:
        print(repr(e))
        return {
            "statusCode": 500,
            "headers": {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*',
                "Access-Control-Allow-Credentials": True
            },
            "body": json.dumps({"error": repr(e)})
        }

In [10]:
answer = question_answering_pipeline(article)

In [11]:
answer

'<pad> a front that had been stalled over the mid-Atlantic lifted north of the region. later that day and into the evening, the front once again approached. low pressure tracked through new England and began to intensify offshore in the Gulf of Maine.</s>'